### Configuracion Inicial

In [ ]:
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

dataset = pd.read_csv("../datasets/properati_argentina_2021.csv")
sns.set()

SEMILLA = 0
TEST_SIZE = 0.2


def obtener_frecuencia_relativa(series):
    frecuencia_absoluta = series.value_counts()
    frecuencia_relativa = frecuencia_absoluta / frecuencia_absoluta.sum()
    return frecuencia_relativa


def separate_date(dataset, column_name):
    dataset[f"{column_name}_year"] = dataset[column_name].str[:4].astype(int)
    dataset[f"{column_name}_month"] = dataset[column_name].str[5:7].astype(int)
    dataset[f"{column_name}_day"] = dataset[column_name].str[8:10].astype(int)
    dataset.drop([column_name], axis=1, inplace=True)


# 1. Análisis Exploratorio y Preprocesamiento de Datos

## Separacion Train-Test

In [ ]:
dataset_train, dataset_test = train_test_split(
    dataset, test_size=TEST_SIZE, random_state=SEMILLA
)

print("Proporcion - Train:", dataset_train.shape[0] / dataset.shape[0])
print("Proporcion - Test:", dataset_test.shape[0] / dataset.shape[0])


## Filtrado del set de Training

Damos un primer vistazo al dataset, para observar las columnas y sus valores

In [ ]:
dataset_train.head(5)


Definimos un diccionario de filtros que vamos a utilizar para filtrar nuestro dataset, y filtramos las columnas que no nos interesan

In [ ]:
filtro_tipo_propiedad = ["casa", "ph", "departamento"]
filtro_operacion = ["venta"]
filtro_ubicacion = ["capital federal"]
filtro_moneda = ["usd"]

filtros = {
    "property_type": filtro_tipo_propiedad,
    "operation": filtro_operacion,
    "place_l2": filtro_ubicacion,
    "property_currency": filtro_moneda,
}


for columna in filtros:
    dataset_train[columna] = dataset_train[columna].str.lower()

for columna_a_filtrar, filtro in filtros.items():
    dataset_train = dataset_train[dataset_train[columna_a_filtrar].isin(filtro)]

dataset_train.shape


Observamos la composicion de datos NULL en las columnas del dataset

In [ ]:
filas_totales = dataset_train.shape[0]
dataset_train.isna().sum() / filas_totales


Las columnas `place_l5` y `place_l6` contienen `NaN` en todas sus filas. Ademas, `place_4` contiene `NaN` en el 96% de sus filas. Por estas razones, eliminamos las columnas.

Las columnas `id` y `property_title` no nos sirves para predecir nada, ya que son valores arbitrarios que no aportan al analisis. Por esta razon, las eliminamos tambien.

Las `property_currency`, `place_l2`, `operation` solo contienen un valor posible debido al filtrado, por esto las eliminamos


In [ ]:
columnas_eliminar = [
    "property_title",
    "id",
    "place_l4",
    "place_l5",
    "place_l6",
    "property_currency",
    "operation",
    "place_l2",
]
dataset_train.drop(columnas_eliminar, axis=1, inplace=True)


`place_l3` no es un nombre muy significativo, lo renombramos a `barrio`

In [ ]:
dataset_train = dataset_train.rename(columns={"place_l3": "barrio"})

Convertimos las fechas en un mejor formato para el analisis.

In [ ]:

dataset_train["start_date"] = pd.to_datetime(
    dataset_train["start_date"], errors="coerce"
)
dataset_train["end_date"] = pd.to_datetime(dataset_train["end_date"], errors="coerce")
dataset_train["created_on"] = pd.to_datetime(
    dataset_train["created_on"], errors="coerce"
)



## Analisis Exploratorio

### Analisis Inicial


Observamos la estructura general del dataset


In [ ]:
dataset_train.shape


In [ ]:
dataset_train.dtypes


In [ ]:
dataset_train.head(5)


Separamos el dataset en variables cualitativas, cuantitativas, y temporales

In [ ]:
columnas_cuantitativas = [
    "latitud",
    "longitud",
    "property_rooms",
    "property_bedrooms",
    "property_surface_total",
    "property_surface_covered",
    "property_price",
]
variables_cuantitativas = dataset_train[columnas_cuantitativas]
variables_cuantitativas.head()


In [ ]:
columnas_temporales = [
    "start_date",
    "end_date",
    "created_on",
]
variables_temporales = dataset_train[columnas_temporales]
variables_temporales.head()


In [ ]:
columnas_cualitativas = ["barrio", "property_type"]
variables_cualitativas = dataset_train[columnas_cualitativas]
variables_cualitativas.head()


### Variables Cuantitativas

Generamos estadisticas descriptivas para las columnas cuantitativas (medidas de resumen)

In [ ]:
variables_cuantitativas.describe()


Graficamos un mapa de correlaciones de las variables cuantitativas

In [ ]:
matriz_correlaciones = variables_cuantitativas.corr()
sns.heatmap(data=matriz_correlaciones, annot=True, fmt=".2f", vmin=-1, vmax=1)


Observamos que las variables con mas relacionadas son:
 - `property_rooms` con `property_bedrooms`
 - `property_surface_covered` con `property_surface_total`.
 - `property_price` con `property_rooms` y `property_bedrooms` (con un porcentaje mucho menor)


Realizamos graficos para observar estas relaciones.

In [ ]:
figs, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = np.ndarray.flatten(axes)

correlaciones = (
    ("property_rooms", "property_bedrooms"),
    ("property_surface_covered", "property_surface_total"),
    ("property_price", "property_rooms"),
    ("property_price", "property_bedrooms"),
)
for (i, correlacion) in enumerate(correlaciones):
    sns.scatterplot(ax=axes[i], data=dataset_train, x=correlacion[0], y=correlacion[1])


Vimos que hay cierta relacion lineal, pero es afectada por el alto numero de valores atipicos. Si los eliminamos rapidamente, el grafico cambia drasticamente. 

Graficamos con un _alpha_ para observar mejor la distribución

In [ ]:
columnas = (
    "property_surface_covered",
    "property_surface_total",
    "property_rooms",
    "property_bedrooms",
    "property_price",
)
sin_atipicos = dataset_train.copy()
for columna in columnas:
    sin_atipicos = sin_atipicos[
        sin_atipicos[columna] < sin_atipicos[columna].quantile(0.75) * 3.5
    ]


figs, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = np.ndarray.flatten(axes)
for (i, correlacion) in enumerate(correlaciones):
    sns.scatterplot(ax=axes[i], data=sin_atipicos, x=correlacion[0], y=correlacion[1], alpha=0.05)


### Variables Cualitativas

Observamos la cantidad de valores posibles de cada variable


In [ ]:
variables_cualitativas.nunique()


Graficamos la distribucion de los barrios

In [ ]:
fig, axes = plt.subplots(
    nrows=1, ncols=2, figsize=(15, 5), gridspec_kw={"width_ratios": [3, 1]}
)

frecuencia_relativa = obtener_frecuencia_relativa(variables_cualitativas["barrio"])
ax = sns.barplot(ax=axes[0], x=frecuencia_relativa.index, y=frecuencia_relativa.values)
ax.set(title="Distribucion de barrios", xlabel="barios", ylabel="frecuencia")
ax.tick_params("x", labelrotation=90)

frecuencia_relativa = obtener_frecuencia_relativa(
    variables_cualitativas["property_type"]
)
ax = sns.barplot(ax=axes[1], x=frecuencia_relativa.index, y=frecuencia_relativa.values)
ax.set(
    title="Distribucion del tipo de propiedad",
    xlabel="tipo de propiedad",
    ylabel="frecuencia",
)


### Variables Temporales

Graficamos la distribución de las variables temporales


In [ ]:
figs, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))

for ax in axes:
    ax.tick_params("x", labelrotation=90)

sns.histplot(ax=axes[0], data=variables_temporales, x="start_date", stat="density").set(
    title="Distribucion de fecha de inicio"
)
sns.histplot(ax=axes[1], data=variables_temporales, x="end_date", stat="density").set(
    title="Distribucion de fecha de finalizacion"
)
sns.histplot(ax=axes[2], data=variables_temporales, x="created_on", stat="density").set(
    title="Distribucion de fecha de creacion"
)


Podemos eliminar `created_on`, ya que tiene distribucion practicamente identica a `start_date`

## Analisis de Valores Atipicos (TODO)

SOLO PARA PROBAR - ACTUALIZAR CON VERDADERO ANALISIS

In [ ]:
for columna in variables_cuantitativas.columns:
    filtro_columna = dataset_train[columna] > dataset_train[columna].quantile(0.99)
    dataset_train[columna] = dataset_train[columna].mask(filtro_columna, np.NaN)

## Analisis de datos faltantes

Graficamos la distribucion de datos nulos

In [ ]:
cantidad_de_datos_nulos_por_columna = dataset_train.isna().sum()
frecuencia_relativa = cantidad_de_datos_nulos_por_columna / dataset_train.shape[0]
frecuencia_relativa = frecuencia_relativa[frecuencia_relativa.values != 0]
frecuencia_relativa = frecuencia_relativa.sort_values(ascending=False)

ax = sns.barplot(x=frecuencia_relativa.index, y=frecuencia_relativa.values)
ax.set(
    title="distribucion de datos faltantes por columna",
    xlabel="columna",
    ylabel="distribucion",
)
ax.tick_params("x", labelrotation=90)


Graficamos la distribucion de cantidad de datos faltantes por fila. 

Vemos que la cantidad de filas con mas de 3 datos faltantes es minima


In [ ]:
distribucion_faltantes_por_fila = (
    dataset_train.isna().sum(axis=1).value_counts() / dataset_train.shape[0]
)
ax = sns.barplot(
    x=distribucion_faltantes_por_fila.index,
    y=distribucion_faltantes_por_fila.values,
)
ax.set(
    xlabel="Cantidad de datos faltantes por fila",
    ylabel="proporcion",
    title="proporcion de datos faltantes por fila",
)


## Imputacion de datos faltante

Algunas estrategias para imputar los datos faltantes:

- Sustitución de Casos: Se reemplaza con valores no observados. Hay que consultar con un experto.
- Sustitución por Media: Se reemplaza utilizando la medida calculada de los valores presentes. Sin embargo, esto trae consecuencias
    - La varianza estimada por la nueva variable no es válida ya que es atenuada por los valores repetidos
    - Se distorsiona la distribución
    - Las correlaciones que se observen estaran deprimidas debido a la repetición de un solo valores constante.
- Imputación Cold Deck: Se pueden obtener los datos faltantes a partir de otras variables del dataset.
- Imputacion Hot Deck: Se reemplazan los datos faltante con los valores que resultan más “similares”. Tenemos que definir que es “similar”, para ello se puede usar la tecnica: K vecinos más cercanos.
- Imputación por Regresión: El dato faltante es reemplaza con el valor predicho por un modelo de regresión.
- MICE (multivariate imputation by chained equations): Trabaja bajo el supuesto de que el origen de los datos es MAR (missing at random). Es un proceso de imputación iterativo, donde cada iteración los valores faltantes se predicen en función de las variables restantes. El proceso se repite hasta que se encuentre consistencia en los datos (usualmente 10 iteraciones es suficiente). La primera iteración se realiza por uno de los métodos vistos anteriormente para rellenar los datos faltantes.

### Imputacion `property_bedrooms`, `property_rooms`, `property_price`

Anteriormente observamos que hay una correlacion entre `property_bedrooms`, `property_rooms` y `property_price`. Podemos entrenar un modelo lineal para predecir los datos faltantes

Imputamos los datos utilizando un imputador iterativo

In [ ]:
columnas_a_imputar = ["property_bedrooms", "property_rooms", "property_price"]

modelo_lineal = LinearRegression()
imp = IterativeImputer(
    estimator=modelo_lineal,
    missing_values=np.nan,
    max_iter=20,
    random_state=SEMILLA,
)

columnas_imputadas = imp.fit_transform(dataset_train[columnas_a_imputar])

dataset_imputado = pd.DataFrame(columnas_imputadas, columns=columnas_a_imputar).astype(
    int
)


Observamos que tienen la misma distribución que los datos originales

In [ ]:
figs, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 16))

ax = sns.histplot(
    ax=axes[0][0], x=dataset_train.property_rooms, discrete=True, stat="density"
)
ax.set(title="distribucion de cantidad de cuartos", xlabel="cantidad de cuartos")

ax = sns.histplot(
    ax=axes[0][1], x=dataset_train.property_bedrooms, discrete=True, stat="density"
)
ax.set(title="distribucion de cantidad de baños", xlabel="cantidad de baños")

ax = sns.kdeplot(ax=axes[0][2], x=dataset_train.property_price)
ax.set(title="distribucion de precio de propiedad", xlabel="precio de propiedad")

ax = sns.histplot(
    ax=axes[1][0], x=dataset_imputado.property_rooms, discrete=True, stat="density"
)
ax.set(
    title="distribucion de cantidad de cuartos (imputado)", xlabel="cantidad de cuartos"
)

ax = sns.histplot(
    ax=axes[1][1], x=dataset_imputado.property_bedrooms, discrete=True, stat="density"
)
ax.set(title="distribucion de cantidad de baños (imputado)", xlabel="cantidad de baños")

ax = sns.kdeplot(ax=axes[1][2], x=dataset_imputado.property_price)
ax.set(
    title="distribucion de precio de propiedad (imputado)", xlabel="precio de propiedad"
)


Actualizamos los valores imputados en el nuevo dataframe

In [ ]:
dataset_train[columnas_a_imputar] = dataset_imputado[columnas_a_imputar].values

dataset_train.isna().sum()


### Imputacion `property_surface_covered`, `property_surface_total`


Tambien observamos que hay una correlacion entre `property_surface_covered`, `property_surface_total`. Podemos entrenar otro modelo lineal para predecir estos datos faltantes

Imputamos los datos utilizando un imputador iterativo

In [ ]:
columnas_a_imputar = ["property_surface_covered", "property_surface_total"]

modelo_lineal = LinearRegression()
imp = IterativeImputer(
    estimator=modelo_lineal,
    missing_values=np.nan,
    max_iter=20,
    verbose=0,
    random_state=SEMILLA,
)

columnas_imputadas = imp.fit_transform(dataset_train[columnas_a_imputar])

dataset_imputado = pd.DataFrame(columnas_imputadas, columns=columnas_a_imputar).astype(
    int
)


Observamos que tienen la misma distribución que los datos originales

In [ ]:
figs, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 16))

ax = sns.kdeplot(ax=axes[0][0], x=dataset_train.property_surface_covered)
ax.set(title="distribucion de superficie cubierta", xlabel="superficie cubierta")

ax = sns.kdeplot(ax=axes[0][1], x=dataset_train.property_surface_total)
ax.set(title="distribucion de superficie total", xlabel="superficie total")

ax = sns.kdeplot(ax=axes[1][0], x=dataset_imputado.property_surface_covered)
ax.set(
    title="distribucion de superficie cubierta (imputado)", xlabel="superficie cubierta"
)

ax = sns.kdeplot(ax=axes[1][1], x=dataset_imputado.property_surface_total)
ax.set(title="distribucion de superficie total (imputado)", xlabel="superficie total")


Actualizamos los valores imputados en el nuevo dataframe

In [ ]:
dataset_train[columnas_a_imputar] = dataset_imputado[columnas_a_imputar].values

dataset_train.isna().sum()


### Imputacion de `latitud`, `longitud`, `barrio`